# Graphics

In [ ]:
import numpy as np
from scipy import misc, ndimage
from skimage import feature, io, filters, morphology, measure
from skimage import data, color
import matplotlib.pyplot as plt
from drawing import *
%matplotlib inline

## What is Computer Graphics?

It is the field of computer science associated with generating and manipulating images. It is used in many fields such as design, movies, video games, etc. Therefore to understand computer graphics we need to understand how images are represented on computers.

## Image Basics
Images are made up of a grid of small colored squares called **Pixels**

In [ ]:
filename = "banana.jpg"
img = data.imread(filename,as_grey=True)
img_rgb = misc.imread(filename)

In [ ]:
plt.imshow(img_rgb)
plt.show()
print("shape of image RGB is " + str(img_rgb.shape))

### RGB Color Model

Images are made up of a 2 dimensional grid. The size of these dimensions are the first 2 numbers in the shape output above.

We call each cell in the grid a *pixel*. Each pixel has a location that is equal to its position in the x and y axis. The top left location is (0, 0)

If we zoom in far enough, we can see these pixels:

In [ ]:
plt.imshow(img_rgb, interpolation='none')
plt.xlim(0,5)
plt.ylim(0,5)
plt.show()

How does an image determine what color to display each pixel? This is what the third dimension of an image is for. Here is an example value of this dimension:

In [ ]:
print(img_rgb[0,0])

Each channel corresponds to the colors Red Green and Blue (where "RGB" comes from), and each value is how much of that color to mix.

In [ ]:
#print(str(img_rgb))

You can change the value of a pixel by setting it to a new combination of Red Green and Blue:

In [ ]:
# Set the new color to (0, 255, 0)
img_rgb[1,1] = np.array([0, 255, 0])

# show the modified image
plt.imshow(img_rgb, interpolation='none')
plt.xlim(0,5)
plt.ylim(0,5)
plt.show()

Now we will write our first piece of computer graphics code.

The following is a python snippet that will go through every pixel in the image and set it to a color based on the x,y position of the pixel. This idea of setting the color of a pixel based on its position in the image is central to computer graphics!

We will also introduce you to some drawing methods that we will be using to generate images.
* **new_drawing(w, h)** - creates a new image for us to draw on with width **w** and height **h**
* **set_color(r, g, b)** - set the color for the next time we use **set_pixel**. Think of it like changing the color of your pen
* **set_pixel(x, y)** - set the pixel at coordinate **x**, **y** to the current value of our pen

This type of graphic is called a gradient. Try to modify this piece of code to generate gradients of different colors.

In [ ]:
width = 200
height = 200
new_drawing(width, height)
for x in range(width):
    for y in range(height):
        set_color(255 * (x / width), 255 * (y / height), 127)
        set_pixel(x, y)
show()

## Graphics Techniques

There are 2 primary ways of doing computer graphics: **Projective geometry**, and **raytracing**. This tutorial will focus on the raytracing technique, but we will discuss projective geometry briefly.

### Projective Geometry

Most modern computers are equippied with video cards that are designed to quickly process projective geometry. Video games and most 3d animations that run on your computer will use this technique. Shapes are represented as 3d triangles, and you must use many (millions) of triangles to make a very nice looking scene. In fact most of the time when you upgrade your video card a common metric is how many triangles per second your card can render.

On the upside, every computer has special hardware to generate these kinds of images so it is very fast. But it is also a lot more complicated and requires special understanding of the video card hardware.

<img src="images/polygon-count.jpeg" />

### Raytracing

The other technique is often used when realtime animation is not needed. Instead of using triangles as its fundamental shape type, it can represent any shape (sphere, boxes, planes, etc).

These types of images are generated by shooting a line outward from every pixel in the image. We write code to find what shape that line intersects with, and we use some shading techniques to determine what the color should be. Then we set the color of the pixel that we shot the line out of to that color. We do this for every pixel.

<img src="images/Ray_trace_diagram.svg" style="width:600px;"/>

We can see from this diagram that every object in the scene has a location in 3d space. This will be important to keep in mind for later!

This process is extremely slow, but its beneficial in that it can generate very realistic images. In fact, all of the special effects in movies, and animated movies like Pixar films, use a raytracer to generate their graphics.

<img src="images/Glasses_800_edit.png" style="width: 600px;"/>

## Writing a Raytracer

The rest of this tutorial will involve a walkthrough of writing our own simple raytracer. First things first, to really understand what is going on it is important to understand some vector math. We have written some functions to help you perform some basic vector math operations. If you arent familiar with this kind of thing, thats ok! Its not completely necessary to get through the tutorial.

In [ ]:
# Useful math functions for dealing with vectors
# Not strictly necessary to understand these
def add(a, b):
    return (a[0] + b[0], a[1] + b[1], a[2] + b[2])

def scale(a, r):
    return (a[0] * r, a[1] * r, a[2] * r)

def dot(a, b):
    return a[0] * b[0] + a[1] * b[1] + a[2] * b[2]

def subtract(a, b):
    return add(a, scale(b, -1))

def normalize(a):
    return scale(a, 1/sqrt(dot(a,a)))

Our raytracer will only initially trace against spheres. Lets define a class in Python that allows us to keep track of these. The important attributes of a sphere will be its world coordinate location (position), its size (radius) and its color.

#### Sphere Class

For every type of object we want to render with the raytracer, we need to define how to intersect it with a ray. We do this by defining the method **intersect** on each shape type. We have provided an implementation of ray-sphere intersection in the Sphere class below. If you are curious about how it works, check out this wikipedia article (there are also many other resources online about the topic)

https://en.wikipedia.org/wiki/Line%E2%80%93sphere_intersection

We also need each type of object to tell the raytracer how to color it. We must define the function **colorize** for each shape type. For now, we will just return the color of the sphere and see what the results look like!

In [ ]:
class Sphere:
    def __init__(self, radius, position, color):
        self.radius = radius
        self.position = position
        self.color = color
    def intersect(self, source, direction, min_distance=0.01):
        v = subtract(source, self.position)
        b = -dot(v, direction)
        v2, r2 = dot(v,v), self.radius * self.radius
        d2 = b*b - v2 + r2
        if d2 > 0:
            for d in (b - sqrt(d2), b+sqrt(d2)):
                if d > min_distance:
                    return d
        return None
    def colorize(self, direction, surface, scene):
        return self.color

Now that we have a shape that we can raytrace, we need to be able to actually trace it! We will make this easier be defining a Scene class

#### Scene Class

The scene class allows us to keep track of all of the objects in the scene, as well as the camera and light positions. We will need attributes to keep track of all of these. Additionally, the Scene class will allow us to trace a line into the scene, to intersect with the objects in front of the line. This is a critical part of how the raytracer works. Please read the comments in the code below to get an understanding of what this code does!

In [ ]:
class Scene:
    def __init__(self, objects, lights, camera, ambient):
        self.objects = objects
        self.lights = lights
        self.camera = camera
        self.ambient = ambient
    def trace(self, source, direction):
        # We will keep track of all of the objects that we hit in this list
        hits = []
        # Go through every object in the scene, and try to intersect it.
        # If we find an intersection, add it to the hit list
        for s in self.objects:
            d = s.intersect(source,direction)
            if d is not None:
                hits.append((d, s))
        # If we have hit nothing, then its just empty space so set the color to black!
        if not hits:
            return (0,0,0)
        # Find the closest object in the hit list
        distance, sphere = min(hits)
        surface = add(source, scale(direction, distance))
        # Return the color for this object
        return sphere.colorize(direction, surface, self)

Now we have most of the components to trace a bunch of shapes in our scene, lets write a function that creates a scene. That is, it will create new instances of Sphere objects and place them in a new Scene object, as well as define all the positions for lights and the camera.

We also define a function here called render, which will create a new image and go through every pixel and trace into the scene from that location. Notice how render looks rather similar to the gradient we generated above? They both involve computing pixel values at every pixel.

In [ ]:
def default_scene():
    # We will create a scene with 1 sphere that is color (255, 255, 255) in RGB.
    objects = [
        Sphere(1, (0,0,5), (255,255,255)),
    ]
    lights = (2,2,0)
    camera = (0,0,1)
    ambient = 0.2
    return Scene(objects, lights, camera, ambient)
    
def render():
    # Make a 200x200 sized image. Try setting this to something larger and see how it takes longer!
    width = 200
    height = 200
    scene = default_scene()
    new_drawing(width, height)
    for x in range(width):
        for y in range(height):
            # The direction that we will trace outward will be based on the pixel location
            direction = normalize((x/width-0.5, y/height-0.5, 1))
            # This is where we trace into the scene to get the color of this pixel
            color = scene.trace(scene.camera, direction)
            # Set the pixel color!
            set_color(color[0], color[1], color[2])
            set_pixel(x, height-y)
    show()

Lets give it a try! What do you expect to see?

In [ ]:
render()

Is this what you expected? Its rather plain, isnt it? This is because in the colorize function, all we did was return the color of the sphere. We did not take the lights into account...

We can modify the colorize function to change how the sphere gets its color. We will take into account the angle to the light that is in the scene object

(See the previous graphic in the Raytracing section to get an idea of how this works)

In [ ]:
def colorize(self, direction, surface, scene):
    to_surface = subtract(surface, self.position)
    to_light = subtract(scene.lights, surface)
    intensity = max(scene.ambient,
                    dot(normalize(to_light), normalize(to_surface)))
    return scale(self.color, intensity)

Sphere.colorize = colorize
render()

That is more like it!

We can also modify the scene to include multiple objects. We do that by redefining the **default_scene** method. Here is an example of that.

In [ ]:
def default_scene():
    # Two spheres now, with colors!
    objects = [
        Sphere(1, (0,0,5), (0,255,255)),
        Sphere(1, (1,0,5), (255,0,255)),
    ]
    lights = (2,2,0)
    camera = (0,0,1)
    ambient = 0.2
    return Scene(objects, lights, camera, ambient)

render()

## What's next?
From here you can modify the ray tracer in countless ways. You could add new material types (reflective, different shading types, glass), you could add casted shadows, new shape types (planes, boxes), you can add textures.

Another option would be to programatically generate a scene. What are we able to render using just spheres? Could we general sphere locations/sizes using an algorithm to create a cool piece of art? Lets see an example of that...

In [ ]:
def default_scene():
    objects = []
    y = -5
    for i in range(1, 10):
        objects.append(Sphere(i/5, (0,y,15), (255-25*i,25*i,0)))
        y += i/5
    lights = (2,2,0)
    camera = (0,0,1)
    ambient = 0.2
    return Scene(objects, lights, camera, ambient)

render()

Some kind of worm?? Who knows!

Here are some ideas of projects you could create with this starter code:

### Project Ideas

* Add some features to the raytracer
    * Reflections, glass objects (quite advanced)
    * Other shapes
    * Textures on the surfaces
* Write code to generate a cool looking scene
    * This can either be realistic or abstract
* If you arent interested in raytracing, you can create other types of images like the gradient